# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

$tf_{i,j} = \frac{n_{i,j}}{\sum_k{n_{k.j}}}$

$idf_i = log\frac{|D|}{|{j:t_i \in d_j }|}$

where $n_{i,j}$ is the frequency of ith word in jth document, $|D|$ is number of total documentations, and $|{j:t_i \in d_j }|$ is the number of document that contains the word

#### 2. LDA算法的基本假设是什么？

- Per-document topic distributions: The topics of each document obey a probability distribution.

- Per-topic word distributions: The words of each topic obey  a probability distribution. 

#### 3. 在TextRank算法中构建图的权重是如何得到的？

将每个句子看成图中的一个节点，若两个句子之间有相似性，认为对应的两个节点之间有一个无向有权边，权值是相似度

#### 4. 什么是命名实体识别？ 有什么应用场景？

命名实体识别是将文本中的元素分成预先定义的类，如人名、地名、 机构名、时间、货币等等。作为自然语言的承载信息单位，命名实体识别属于文本信息处理的基础的研究领域，是信息抽取、信息检索、机器翻译、问答系统等多种自然语言处理技术中必不可少的组成部分。

#### 5.NLP主要有哪几类任务 ？

1. 序列标注：分词/POS Tag/NER/语义标注
2. 分类任务：文本分类/情感计算
3. 句子关系判断：Entailment/QA/自然语言推理
4. 生成式任务：机器翻译/文本摘要

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [1]:
import re
import sys
from operator import itemgetter
from collections import defaultdict
import jieba.posseg
import pandas as pd

In [2]:
news = pd.read_csv("../新华社数据.csv", encoding='gb18030')["content"][:100]
sentences_list = []
for new in news:
    new = new.replace("\r\n", "")
    for sent in new.split("。"):
        sent = sent.strip()
        if len(sent) > 4:
            sentences_list.append(sent)
sentences = " ".join(sentences_list)
sentences[:100]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作 有人猜测这也是将精力主要用到M'

In [3]:
class UndirectWeightedGraph:
    d = 0.85
 
    def __init__(self):
        self.graph = defaultdict(list)
 
    def addEdge(self, start, end, weight):
        # use a tuple (start, end, weight) instead of a Edge object
        self.graph[start].append((start, end, weight))
        self.graph[end].append((end, start, weight))
 
    def rank(self):
        ws = defaultdict(float)
        outSum = defaultdict(float)
 
        wsdef = 1.0 / (len(self.graph) or 1.0)
        for n, out in self.graph.items():
            ws[n] = wsdef
            outSum[n] = sum((e[2] for e in out), 0.0)
 
        # this line for build stable iteration
        sorted_keys = sorted(self.graph.keys())
        for x in range(10):  # 10 iters
            for n in sorted_keys:
                s = 0
                for e in self.graph[n]:
                    s += e[2] / outSum[e[1]] * ws[e[1]]
                ws[n] = (1 - self.d) + self.d * s
 
        (min_rank, max_rank) = (sys.float_info[0], sys.float_info[3])
 
        for w in ws.values():
            if w < min_rank:
                min_rank = w
            if w > max_rank:
                max_rank = w
 
        for n, w in ws.items():
            # to unify the weights, don't *100.
            ws[n] = (w - min_rank / 10.0) / (max_rank - min_rank / 10.0)
 
        return ws
 

class TextRank():
 
    def __init__(self):
        self.tokenizer = self.postokenizer = jieba.posseg.dt
        self.span = 5
 
    def pairfilter(self, wp):
        return len(wp.word.strip()) >= 2
 
    def textrank(self, sentence, topK=20, withWeight=True,):
        g = UndirectWeightedGraph()
        cm = defaultdict(int)
        words = tuple(self.tokenizer.cut(sentence))
        for i, wp in enumerate(words):
            if self.pairfilter(wp):
                for j in range(i + 1, i + self.span):
                    if j >= len(words):
                        break
                    if not self.pairfilter(words[j]):
                        continue
                    cm[(wp.word, words[j].word)] += 1
 
        for terms, w in cm.items():
            g.addEdge(terms[0], terms[1], w)
        nodes_rank = g.rank()
        if withWeight:
            tags = sorted(nodes_rank.items(), key=itemgetter(1), reverse=True)
        else:
            tags = sorted(nodes_rank, key=nodes_rank.__getitem__, reverse=True)
 
        if topK:
            return tags[:topK]
        else:
            return tags
 

In [4]:
TextRank().textrank(sentences)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ns/kcb8yjxs6qx72w25lyg4m_jc0000gn/T/jieba.cache
Loading model cost 0.815 seconds.
Prefix dict has been built successfully.


[('市场', 1.0),
 ('中国', 0.7492183274636033),
 ('我们', 0.7390225823952734),
 ('已经', 0.6816553844646037),
 ('进行', 0.6631682441471889),
 ('一个', 0.6327811875513467),
 ('可以', 0.5952540254557545),
 ('企业', 0.5855450134941426),
 ('以及', 0.5701328322041872),
 ('记者', 0.5466108676826839),
 ('目前', 0.527807516555052),
 ('融资', 0.5170155602716184),
 ('乐视', 0.509356316891696),
 ('可能', 0.4565071424754822),
 ('没有', 0.45561089947409117),
 ('冰架', 0.451081458299052),
 ('手机', 0.44503775258642686),
 ('其中', 0.42845878375930074),
 ('内容', 0.4256450315506142),
 ('相关', 0.4170464601023033)]

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。